# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298559605568                   -0.85    4.5
  2   -8.300215417449       -2.78       -1.25    1.0
  3   -8.300447593613       -3.63       -1.87    4.2
  4   -8.300462649618       -4.82       -2.75    2.0
  5   -8.300464434378       -5.75       -3.13    4.4
  6   -8.300464538377       -6.98       -3.29    7.9
  7   -8.300464589112       -7.29       -3.43    2.5
  8   -8.300464619369       -7.52       -3.57    1.0
  9   -8.300464635414       -7.79       -3.73    1.5
 10   -8.300464641867       -8.19       -3.92    1.0
 11   -8.300464643839       -8.71       -4.30    1.4


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67482309009                   -0.70    6.5
  2   -16.67876253265       -2.40       -1.14    1.0
  3   -16.67922603082       -3.33       -1.86    3.4
  4   -16.67928063103       -4.26       -2.77    3.5
  5   -16.67928604041       -5.27       -3.20    4.4
  6   -16.67928621618       -6.76       -3.50    3.6
  7   -16.67928622228       -8.21       -3.98    1.0
  8   -16.67928622420       -8.72       -4.60    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32529141970                   -0.56    6.5
  2   -33.33275704654       -2.13       -1.00    1.2
  3   -33.33589007296       -2.50       -1.68    3.4
  4   -33.33686989214       -3.01       -2.36    4.6
  5   -33.33693565628       -4.18       -2.83    9.1
  6   -33.33694384320       -5.09       -3.76    3.0
  7   -33.33694391590       -7.14       -4.12   12.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298542872202                   -0.85    4.0
  2   -8.300264846660       -2.76       -1.59    1.5
  3   -8.300356991896       -4.04       -2.22   10.2
  4   -8.300326151769   +   -4.51       -2.18    3.4
  5   -8.300464347497       -3.86       -3.67    1.5
  6   -8.300464606944       -6.59       -3.99    6.6
  7   -8.300464641546       -7.46       -4.53    3.4


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32496861835                   -0.56    6.0
  2   -33.32260203807   +   -2.63       -1.27    1.0
  3   -20.68751937834   +    1.10       -0.52    8.0
  4   -33.28713278124        1.10       -1.53    6.4
  5   -33.18740233366   +   -1.00       -1.32    4.0
  6   -28.73928361517   +    0.65       -0.74    4.8
  7   -33.33659234211        0.66       -2.29    6.4
  8   -33.33605191006   +   -3.27       -2.26    3.6
  9   -33.33560025394   +   -3.35       -2.28    3.0
 10   -33.33586836524       -3.57       -2.52    2.5
 11   -33.33688723268       -2.99       -3.05    2.6
 12   -33.33691423550       -4.57       -3.23    3.1
 13   -33.33693782706       -4.63       -3.44    2.6
 14   -33.33693980479       -5.70       -3.54    2.6
 15   -33.33694330564       -5.46       -4.03    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.